<!--NAVIGATION-->
< [Calculate Flow Direction](Flowdirection.ipynb) | [Contents](ContentsCatchmentDelineation.ipynb) | [Catchment Delineation](CatchmentDelineation.ipynb) >

# 6. Delineate streams
In this step we're going to delineate the streams. There are two methods that we're going to apply:
* Using Strahler orders
* Using flow accumulation

## 6.1 Strahler method
Let's start with deriving the Strahler orders using the <code>streamorder</code> operation.
The <code>streamorder</code> operation needs the flow direction map created in the previous section as an input. A detailed description of the <code>streamorder</code> operation can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_streamorder.html).

In [ ]:
import os
from pcraster import *
os.chdir('./Data/')

FlowDirection = readmap('flowdir.map')
StrahlerOrders = streamorder(FlowDirection)
aguila(StrahlerOrders)

Change the legend in Aguila to a ramp from yellow to blue, so the higher orders are blue, like rivers.

The data type of the Strahler order map is ordinal, starting at 1. Now we need to determine after which Strahler order we consider the the flow big enough to call it a river. We do that through calibration. Let's calculate maps with Strahler order 1 to the maximum and compare them with OpenStreetMap.
First we determine the maximum Strahler order by using the <code>mapmaximum</code> operation. The <code>mapmaximum</code> operation is a global operation. More info can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_mapmaximum.html).

In [ ]:
MaximumStrahlerOrder = mapmaximum(StrahlerOrders)

Visualise the result with Aguila.

The problem is that the result is a PCRaster map with for each pixel the maximum value of the map. In order to iterate over the Strahler orders we need to get the value. We can use the <code>cellvalue</code> operation. The <code>cellvalue</code> operation needs the raster for which it has the give the cell value as input as well as a row and column index number. Here all cells are the same, so we can simply use 0 for the index, refering to the first row and first column of the input raster.

In [ ]:
MaximumStrahlerOrderTuple = cellvalue(MaximumStrahlerOrder,0,0)
print(MaximumStrahlerOrderTuple)

The <code>cellvalue</code> operation returns a tuple with two elements: the first is the cell value, the second is a boolean value which shows whether the first element, is valid or not. If the second element is False, the cell contains a missing value. So if we want to have the value we need to take the first element of the tuple.

In [ ]:
MaximumStrahlerOrderValue = MaximumStrahlerOrderTuple[0]
print(MaximumStrahlerOrderValue)

Now we can loop over the Strahler orders and save each map larger or equal to that order.

In [ ]:
for order in range (1,MaximumStrahlerOrderValue + 1):
    Stream = ifthen(StrahlerOrders >= order, boolean(1))
    report(Stream,'stream'+str(order)+'.map')

Visualise the different results and choose which best matches with OpenStreetMap or Google Satellite. You can load the layers in QGIS to do this.

## 6.2 Flow accumulation method
In order to estimate where the river is we can also use the flow accumulation method. In PCRaster that's done with the [accuflux](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_accuflux.html) operation. Accuflux will accumulate material (second argument) over the flow direction. Here we choose 1 for all cells as material.

In [ ]:
FlowAccumulation = accuflux(FlowDirection,1)

Visualise the result with Aguila. Change the legend to shifted logarithmic, because of the large range of values.

In the same way as with the Strahler method we need to determine the minimum flow accumulation that we consider a river. We do this again by comparing boolean maps with a reference map. Let's start wit 9000 pixels.

In [ ]:
RiverFlow = ifthen(FlowAccumulation > 9000, boolean(1))

Try this for different values and check how well it matches with for example OpenStreetMap in QGIS.

After calibration, write the one that fits best to disk with `report`.

The next step is to delineate the catchment belonging to a selected outlet.

<!--NAVIGATION-->
< [Calculate Flow Direction](Flowdirection.ipynb) | [Contents](ContentsCatchmentDelineation.ipynb) | [Catchment Delineation](CatchmentDelineation.ipynb) >